<a href="https://colab.research.google.com/github/sidms24/internship/blob/main/week%205/Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import os
from google.colab import userdata
import requests
import torch
import torch.nn as nn
import Bio
from Bio import SeqIO
token = userdata.get('GITHUB_TOKEN')

In [ ]:

token = userdata.get('GITHUB_TOKEN')
url = "https://raw.githubusercontent.com/sidms24/internship/refs/heads/main/week%205/latents.csv"
filename = "latent.csv"
curl_output_latents = os.system(f"curl -H 'Authorization: token {token}' {url} > {filename}")
print(f"Curl output for latents.csv: {curl_output_latents}")
latents = pd.read_csv("latent.csv", index_col=0)

gp_url = "https://raw.githubusercontent.com/sidms24/internship/refs/heads/main/week%205/Gp_ili_consult_100k.csv"
gp_filename = "gp.csv"
curl_output_gp = os.system(f"curl -H 'Authorization: token {token}' {gp_url} > {gp_filename}")
print(f"Curl output for gp.csv: {curl_output_gp}")
gp = pd.read_csv("gp.csv", skiprows = 4)

seq_url = "https://raw.githubusercontent.com/sidms24/internship/refs/heads/main/week%205/sequences(all).csv"
seq_filename = "sequences.csv"

try:
    response = requests.get(seq_url, headers={'Authorization': f'token {token}'})
    response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    with open(seq_filename, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {seq_filename} successfully.")
except requests.exceptions.RequestException as e:
    print(f"Error downloading {seq_filename}: {e}")


encoder_url = "https://raw.githubusercontent.com/sidms24/internship/refs/heads/main/week%205/encoder_weights.pth"
encoder_filename = "encoder.pth"
os.system(f"curl -H 'Authorization: token {token}' {encoder_url} > {encoder_filename}")

sequences_all_url = "https://raw.githubusercontent.com/sidms24/internship/refs/heads/main/week%205/sequences(all).fasta"
sequences_all_filename = "sequences_all.fasta"
try:
    response = requests.get(sequences_all_url, headers={'Authorization': f'token {token}'})
    response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    with open(sequences_all_filename, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {sequences_all_filename} successfully.")
except requests.exceptions.RequestException as e:
    print(f"Error downloading {sequences_all_filename}: {e}")




Curl output for latents.csv: 0
Curl output for gp.csv: 0
Downloaded sequences.csv successfully.
Downloaded sequences_all.fasta successfully.


In [690]:
seq = pd.read_csv("sequences.csv")
seq.columns
seq.isna().sum()
s = seq.drop(columns = ['Isolate', 'Tissue_Specimen_Source'], axis = 1)
s['Collection_Date'] = pd.to_datetime(s['Collection_Date'], errors='coerce')
s['Release_Date'] = pd.to_datetime(s['Release_Date'] )

def get_season(date):
  if date.month >= 9:
    return f"{date.year}/{str(date.year + 1)[-2:]}"
  elif date.month <= 8:
    return f"{date.year - 1}/{str(date.year)[-2:]}"
  else:
    return np.nan

s['season'] = s['Collection_Date'].apply(get_season)
s['year'] =  s['Collection_Date'].apply(lambda x: x.date().year)

for i in range(len(s)):
  l = s.iloc[i, 1].split(" ")
  a = s.iloc[i, 3]
  if len(l) > 3 and a is np.nan:
    o = l[-1].split("(")[-1]
    s.iloc[i,3] = o.strip(')')
s.dropna(inplace=True)
s.reset_index(drop=True, inplace=True)

In [509]:
def one_hot_encode(seq,max_length = 2366):
    mapping = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], "T": [0, 0, 0, 1]}
    one_hot = [mapping.get(base, [0, 0, 0, 0]) for base in seq]

    # 2. Calculate the amount of padding needed
    padding_needed = max_length - len(one_hot)

    # 3. Create the padding (a list of zero vectors)
    #    Ensure padding_needed is not negative if a sequence is too long.
    if padding_needed > 0:
        padding = [[0, 0, 0, 0]] * padding_needed
        one_hot.extend(padding)

    # Optional: Truncate sequences that are longer than max_length
    elif padding_needed < 0:
        one_hot = one_hot[:max_length]

    return one_hot
l = []
for record in SeqIO.parse("sequences_all.fasta", "fasta"):
  l.append({"Accession": record.id.split('.')[0], "Sequence": str(record.seq)})
seq2 = pd.DataFrame(l)
s = pd.merge(s, seq2, on = "Accession", how = "inner")
s["sequence_one_hot"] = s["Sequence"].apply(one_hot_encode)

In [511]:
class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, latent_dim):
     super().__init__()
     self.encoder = nn.Sequential(
         nn.Linear(input_dim, hidden_dim),
         nn.ReLU(),
         nn.Linear(hidden_dim, hidden_dim // 2),
         nn.ReLU(),
         nn.Linear(hidden_dim // 2, hidden_dim // 4),
         nn.ReLU(),
         nn.Linear(hidden_dim // 4, hidden_dim // 8),
         nn.ReLU(),
         nn.Linear(hidden_dim // 8, latent_dim))

  def forward(self, x):
    return self.encoder(x)

input_dim = 2366 * 4  # Based on the one-hot encoding length and number of bases
hidden_dim = 128 # Set hidden_dim to 128 based on the error message
latent_dim = 2    # Based on Latent_dim1 and Latent_dim2

X= np.array([np.array(seq) for seq in s['sequence_one_hot'].values])
X= np.array([seq.flatten() for seq in X])
X = torch.tensor(X, dtype=torch.float32)

model = Encoder(input_dim, hidden_dim, latent_dim)
model.load_state_dict(torch.load("encoder.pth", map_location=torch.device('cpu')))
model.eval()
with torch.no_grad():
  latent = model(X)


In [691]:
latent = latent.cpu().numpy()
s['Latent'] = latent.tolist()
s["latent_dim1"] = s["Latent"].apply(lambda x: x[0])
s["latent_dim2"] = s["Latent"].apply(lambda x: x[1])
s["latent_dim"] =  s["Latent"].apply(lambda x: np.sqrt((x[0]**2) + (x[1]**2)))

In [692]:
s2 = s
s = s[s['Genotype'] != "H3"]
s = s[s['Genotype'] != "unknown"]


In [338]:
mortality = pd.read_csv("Trends (2).csv")
mortality = mortality[["Time period Sortable","Value"]]
mortality["year"] = mortality['Time period Sortable'].apply(lambda x:int(x /10000))
mortality = mortality.drop("Time period Sortable", axis=1)
mortality = mortality.set_index('year')

In [729]:
import statsmodels.formula.api as smf
import statsmodels.api as sm


s1 = s2.groupby(['year', 'Genotype', 'Segment'])[['latent_dim']].expanding().mean()
s1 = s1.reset_index()
s1 = pd.merge(s1, mortality, on='year', how = 'inner')
s1 = s1.drop(columns = ["level_3"])


s1["log_value"] = np.log(s1['Value'])

X = s1.drop(columns=['Value', 'log_value'])
X = pd.get_dummies(X, columns=['Segment', 'Genotype'], drop_first=True, dtype=int)
y = s1["log_value"]


X_diff = X.diff().dropna()
y_diff = y.diff().dropna()

X_diff, y_diff = X_diff.align(y_diff, join='inner', axis=0)

model = sm.OLS(y_diff, X_diff).fit(cov_type='HC1')


print(model.summary())

# kinda make sense the

                                 OLS Regression Results                                
Dep. Variable:              log_value   R-squared (uncentered):                   0.582
Model:                            OLS   Adj. R-squared (uncentered):              0.581
Method:                 Least Squares   F-statistic:                              4659.
Date:                Thu, 10 Jul 2025   Prob (F-statistic):                        0.00
Time:                        14:52:00   Log-Likelihood:                          34319.
No. Observations:                6902   AIC:                                 -6.861e+04
Df Residuals:                    6888   BIC:                                 -6.851e+04
Df Model:                          14                                                  
Covariance Type:                  HC1                                                  
                       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------